In [5]:
import pandas as pd
import numpy as np
import re

# 검색할 목록
data = pd.read_csv("laundry_data.csv", encoding="CP949")
#landry_name = list("서울특별시 " + data["사업장명"])
laundry_address = list(data["address"])
laundry_name = list(data["name"])
    
params = []

# 정규 표현식
p = re.compile('([가-힣]+(시|군|구|동))')
re = p.findall(laundry_address[0])
# print(re[1][0])
    
for index, ad in enumerate(laundry_address):
    re = p.findall(ad)
    params.append(re[0][0] + " " + f"{laundry_name[index]}")

print(params)

# # 검색할 목록
# data = pd.read_csv("laundry_data.csv", encoding="CP949")
# #landry_name = list("서울특별시 " + data["사업장명"])
# laundry_address = list(data["address"])
# laundry_name = list(data["name"])
    
# params = []

# # 정규 표현식
# p = re.compile('([가-힣]+(시|군|구|동))')
# re = p.findall(laundry_address[0])
# # print(re[1][0])
    
# for index, ad in enumerate(laundry_address):
#     re = p.findall(ad)
#     params.append(re[0][0] + " " + f"{laundry_name[index]}")
        
result_text = []

['종로구 크린토피아 코인워시 독립문현대점', '종로구 빨래터', '종로구 코인워시24빨래터 창신점', '종로구 펭귄하우스 숭인점', '종로구 코인빨래방', '종로구 코인워시24 셀프빨래방 성균관대점', '종로구 코인워시24 광화문점', '종로구 워시프렌즈 서울혜화점', '종로구 AMPM워시큐 누하점', '종로구 크린업24 경복궁역점', '종로구 스피드24클리닝', '종로구 코인워시24셀프빨래방 종로점', '종로구 셀프빨래방24 경복궁점', '종로구 화이트365 성대점', '종로구 이지워시 교남점', '중구 크린토피아 코인워시 신당약수점', '중구 화이트365 신당역점', '중구 화이트365 신당점', '중구 워시엔조이 황학동점', '중구 크린토피아 코인워시 중구동국대점', '중구 동국셀프빨래방', '중구 24시셀프빨래방', '중구 코인워시24시빨래터 서부역점', '중구 워시프렌즈 셀프빨래방 신당상왕십리점', '중구 AMPM워시큐 청계블루', '중구 워시프렌즈 장충동국대점', '중구 셀프빨래방', '종로구 코인빨래방', '종로구 빨래터', '종로구 코인워시24빨래터 창신점', '용산구 크린토피아 코인워시365 이태원보광점', '용산구 크린토피아 코인워시365 용문시장점', '용산구 화이트365 숙대입구역점', '용산구 크린토피아 코인워시365 용산해방촌점', '용산구 워시엔조이 용산남영점', '용산구 크린토피아 코인워시365 이촌2동점', '용산구 워시엔조이 경리단길점', '용산구 화이트365 이태원해방촌점', '용산구 워시피플 후암동108계단점', '용산구 24시셀프빨래방플러스 한남점', '용산구 셀프빨래방 후암점', '용산구 코인워시24빨래터 숙대점', '용산구 더화이트셀프빨래방 원효로점', '용산구 펭귄하우스 용산트럼프월드점', '용산구 펭귄하우스 래미안용산점', '성동구 크린토피아 코인워시 서울숲힐스테이트점', '성동구 크린토피아 코인워시 왕십리역점', '성동구 크린토피아 코인워시 왕십리텐즈힐점', '성동구 크린토피아 코인워시365 왕십리센트라스점', 

In [7]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import re

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    
    
    place_infos = ['종로구 코인빨래방', '종로구 코인워시24 셀프빨래방 성균관대점', '종로구 코인워시24 광화문점', '종로구 워시프렌즈 서울혜화점']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver
# xpath
#     //*[@id="search.keyword.query"] : 검색창 입력
#    //*[@id="search.keyword.submit"] : 검색창 버튼

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                result_text.append(val)
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

##### 0
#### 크린토피아 코인워시 독립문현대점
무악현대아파트 상가 1층에 위치하고 있습니다. 주차는 상가주차장 1시간 무료이구요.^^ 저렴하게 잘 이용하고 있습니다./5
no review in crawling
no review in crawling
#### 빨래터
건조기 짱/5
회원가입하면 포인트 적립되서 좋아요. 이불 빨래 하기 좋더라구요./5
no review in crawling
no review in crawling
#### 코인워시24빨래터 창신점
no review in extract
no review in crawling
no review in crawling


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="info.search.place.list"]/li[4]/div[5]/div[4]/a[1]"}
  (Session info: headless chrome=91.0.4472.124)


In [ ]:
print(result_text)

